In [ ]:
import seaborn as sns
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels
import matplotlib.pyplot as plt
import hyppo
import scipy as sp
import sklearn as sk
from hyppo.tools.cate_sims import (
    heteroskedastic_sigmoidal_sim,
    sigmoidal_sim,
    kclass_sigmoidal_sim,
    nonmonotone_sim,
    simulate_covars
)
from hyppo.causal import GeneralisedPropensityModel
import matplotlib.gridspec as gridspec
import warnings
import os

os.makedirs('../Figures/Appendix/', exist_ok=True)
os.makedirs('../Figures/Temporary/', exist_ok=True)

In [ ]:
# code taken from https://stackoverflow.com/questions/35042255/how-to-plot-multiple-seaborn-jointplot-in-subplot
class SeabornFig2Grid():

    def __init__(self, seaborngrid, fig,  subplot_spec):
        self.fig = fig
        self.sg = seaborngrid
        self.subplot = subplot_spec
        if isinstance(self.sg, sns.axisgrid.FacetGrid) or \
            isinstance(self.sg, sns.axisgrid.PairGrid):
            self._movegrid()
        elif isinstance(self.sg, sns.axisgrid.JointGrid):
            self._movejointgrid()
        self._finalize()

    def _movegrid(self):
        """ Move PairGrid or Facetgrid """
        self._resize()
        n = self.sg.axes.shape[0]
        m = self.sg.axes.shape[1]
        self.subgrid = gridspec.GridSpecFromSubplotSpec(n,m, subplot_spec=self.subplot)
        for i in range(n):
            for j in range(m):
                self._moveaxes(self.sg.axes[i,j], self.subgrid[i,j])

    def _movejointgrid(self):
        """ Move Jointgrid """
        h= self.sg.ax_joint.get_position().height
        h2= self.sg.ax_marg_x.get_position().height
        r = int(np.round(h/h2))
        self._resize()
        self.subgrid = gridspec.GridSpecFromSubplotSpec(r+1,r+1, subplot_spec=self.subplot)

        self._moveaxes(self.sg.ax_joint, self.subgrid[1:, :-1])
        self._moveaxes(self.sg.ax_marg_x, self.subgrid[0, :-1])
        self._moveaxes(self.sg.ax_marg_y, self.subgrid[1:, -1])

    def _moveaxes(self, ax, gs):
        #https://stackoverflow.com/a/46906599/4124317
        ax.remove()
        ax.figure=self.fig
        self.fig.axes.append(ax)
        self.fig.add_axes(ax)
        ax._subplotspec = gs
        ax.set_position(gs.get_position(self.fig))
        ax.set_subplotspec(gs)

    def _finalize(self):
        plt.close(self.sg.fig)
        self.fig.canvas.mpl_connect("resize_event", self._resize)
        self.fig.canvas.draw()

    def _resize(self, evt=None):
        self.sg.fig.set_size_inches(self.fig.get_size_inches())

def make_subplot(df_dat, df_true, title=""):
    fig = plt.figure(constrained_layout=True);
    widths = [2, .5]
    heights = [.5, 2]
    spec = fig.add_gridspec(ncols=2, nrows=2, width_ratios=widths, height_ratios=heights);
    
    ax = fig.add_subplot(spec[0, 0]);
    palette=sns.color_palette("deep", 3)
    g = sns.histplot(data=df_dat, x="Covariate", ax=ax, hue="Class", palette=palette, legend=False, binwidth=0.1)
    g.set(xlabel=None, xticks=[], yticks=[], ylabel=None, xlim=(-1, 1))
    ax = fig.add_subplot(spec[1, 0]);
    g = sns.scatterplot(data=df_dat, x="Covariate", y="Measurement", hue="Class", ax=ax, alpha=0.5, palette=palette)
    ax.set_xlim(-1, 1)
    
    ylim = (np.min((np.min(df_true["Measurement"]), np.min(df_dat["Measurement"]))), np.max((np.max(df_true["Measurement"]), np.max(df_dat["Measurement"]))))
    sns.lineplot(data=df_true, x="Covariate", y="Measurement", hue="Class", ax=ax, linewidth=3, palette=palette, legend=False)
    ax = fig.add_subplot(spec[1, 1]);
    g = sns.histplot(data=df_dat, y="Measurement", ax=ax, hue="Class", palette=palette, legend=False, bins=20, orientation="horizontal")
    g.set(xlabel=None, xticks=[], yticks=[], ylabel=None, ylim=ylim)
    
    return fig
    

# Figure 1

In [ ]:
n = 2000
p=10
pref = 0
sims = {"Heteroskedastic" : heteroskedastic_sigmoidal_sim, "Sigmoidal": sigmoidal_sim, 
        "Non-Monotone": nonmonotone_sim, "K-Class": kclass_sigmoidal_sim}

data_samples = []
data_true = []
for simn, sim_fn in sims.items():
    sim = sim_fn(n, p, eff_sz=1, rotate=False, err=1, balance=.4)
    Ys = sim["Ys"]; Ts = sim["Ts"]; Xs = sim["Xs"]
    true_t = sim["Ttrue"]; true_y = sim["Ytrue"]; true_x = sim["Xtrue"]
    df = pd.DataFrame({"ID" : list(range(0, n)), "Group": Ts, "Outcome" : Ys[:,pref].flatten(), "Covariate" : Xs.flatten(), "Balance":"Unmatched", "Simulation":simn})
    df_true = pd.DataFrame({"Group" : true_t, "Outcome" : true_y[:, pref], "Covariate" : true_x, "Simulation":simn})
    
    balanced_ids = GeneralisedPropensityModel().fit_and_vector_match(Ts, Xs)
    Xs_bal = Xs[balanced_ids]; Ts_bal = Ts[balanced_ids]; Ys_bal = Ys[balanced_ids,:]
    df_bal = pd.DataFrame({"ID": np.array(balanced_ids), "Group": Ts_bal.flatten(), "Outcome" : Ys_bal[:,pref].flatten(), "Covariate" : Xs_bal.flatten(), "Balance":"Matched", "Simulation":simn})
    
    data_samples.append(pd.concat((df, df_bal)))
    data_true.append(df_true)

data_samples = pd.concat(data_samples)
data_true = pd.concat(data_true)

In [ ]:
def make_subplot(df_dat, df_true, samples=None, title="", rsamp=None, ylim=(-5, 5)):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        
        if samples is None:
            samples = range(0, df_dat.shape[0])
        
        fig = plt.figure(constrained_layout=True, figsize=(3, 2.5))
        widths = [2, .5]
        heights = [.5, 2]
        spec = fig.add_gridspec(ncols=2, nrows=2, width_ratios=widths, height_ratios=heights)
        
        ax = fig.add_subplot(spec[0, 0])
        palette=sns.color_palette("deep", 3)
        g = sns.histplot(data=df_dat, x="Covariate", ax=ax, hue="Group", stat="density", common_norm=False, palette=palette, legend=False, bins=30)
        g.set(xlabel=None, xticks=[], yticks=[], ylabel=None, xlim=(-1, 1))
        ax = fig.add_subplot(spec[1, 0])
        g = sns.scatterplot(data=df_dat[np.in1d(df_dat["ID"], samples)], x="Covariate", y="Outcome", hue="Group", ax=ax, alpha=0.5, legend=False, palette=palette)
        ax.set_xlim(-1, 1)
        ax.set(ylim=ylim)
        
        sns.lineplot(data=df_true, x="Covariate", y="Outcome", hue="Group", ax=ax, linewidth=3, palette=palette, legend=False)
        ax = fig.add_subplot(spec[1, 1])
        g = sns.histplot(data=df_dat, y="Outcome", ax=ax, hue="Group", stat="density", common_norm=False, palette=palette, legend=False, bins=30, orientation="horizontal")
        g.set(xlabel=None, xticks=[], yticks=[], ylabel=None)
        
        return fig

In [ ]:
samples = np.random.choice(n, 100, replace=False)
g11 = make_subplot(data_samples[(data_samples["Simulation"] == "Heteroskedastic") & (data_samples["Balance"] == "Unmatched")],
                   data_true[data_true["Simulation"] == "Heteroskedastic"], samples=samples, ylim=(-4, 10))
g11.savefig("../Figures/Temporary/Heteroum.pdf")
g21 = make_subplot(data_samples[(data_samples["Simulation"] == "Heteroskedastic") & (data_samples["Balance"] == "Matched")],
                   data_true[data_true["Simulation"] == "Heteroskedastic"], samples=samples, ylim=(-4, 10))
g21.savefig("../Figures/Temporary/Heterom.pdf")

In [ ]:
g21.savefig("../Figures/Temporary/Difffnm.pdf")
g11 = make_subplot(data_samples[(data_samples["Simulation"] == "Sigmoidal") & (data_samples["Balance"] == "Unmatched")],
                   data_true[data_true["Simulation"] == "Sigmoidal"], samples=samples, ylim=(-2, 12))
g11.savefig("../Figures/Temporary/Sigmoidalum.pdf")
g21 = make_subplot(data_samples[(data_samples["Simulation"] == "Sigmoidal") & (data_samples["Balance"] == "Matched")],
                   data_true[data_true["Simulation"] == "Sigmoidal"], samples=samples, ylim=(-2, 12))
g21.savefig("../Figures/Temporary/Sigmoidalm.pdf")
g12 = make_subplot(data_samples[(data_samples["Simulation"] == "Non-Monotone") & (data_samples["Balance"] == "Unmatched")],
                   data_true[data_true["Simulation"] == "Non-Monotone"], samples=samples, ylim=(-4, 4))
g12.savefig("../Figures/Temporary/Nonmonotoneum.pdf")
g22 = make_subplot(data_samples[(data_samples["Simulation"] == "Non-Monotone") & (data_samples["Balance"] == "Matched")],
                   data_true[data_true["Simulation"] == "Non-Monotone"], samples=samples, ylim=(-4, 4))
g22.savefig("../Figures/Temporary/Nonmonotonem.pdf")
g13 = make_subplot(data_samples[(data_samples["Simulation"] == "K-Class") & (data_samples["Balance"] == "Unmatched")],
                   data_true[data_true["Simulation"] == "K-Class"], samples=samples, ylim=(-2, 12))
g13.savefig("../Figures/Temporary/Kclasssum.pdf")
g23 = make_subplot(data_samples[(data_samples["Simulation"] == "K-Class") & (data_samples["Balance"] == "Matched")],
                   data_true[data_true["Simulation"] == "K-Class"],samples=samples,  ylim=(-2, 12))
g23.savefig("../Figures/Temporary/Kclassm.pdf")

###### Supplement Figures

## Covariate Construction and Matching

In [ ]:
n = 500000
pi = 0.5
Ts = np.random.binomial(1, pi, size=n)
Xs = simulate_covars(Ts, balance=0.4)
fig, axs = plt.subplots(1, 2, figsize=(10, 4))
palette=sns.color_palette("deep", 3)
sns.histplot(data=pd.DataFrame({"Covariate": Xs, "Group": Ts}), x="Covariate", hue="Group", fill="Group", stat="proportion", palette=palette, bins=30, ax=axs[0])
axs[0].set(xlim=(-1, 1), title="(A) Covariate Distribution, $\\pi_b = 0.4$\nBefore Vector Matching")
xmin_loc = np.min(Xs[Ts == 1])
xmax_loc = np.max(Xs[Ts == 0])
axs[0].axvline(x=xmin_loc, color="red", linestyle="--", linewidth=2)
axs[0].axvline(x=xmax_loc, color="red", linestyle="--", linewidth=2)
axs[0].axvline(x=0, color="black", linewidth=2)
balance_ids = GeneralisedPropensityModel().fit_and_vector_match(Ts, Xs)
Xs_bal = Xs[balance_ids]; Ts_bal = Ts[balance_ids]
sns.histplot(data=pd.DataFrame({"Covariate": Xs_bal, "Group": Ts_bal}), x="Covariate", hue="Group", fill="Group", palette=palette, stat="proportion", bins=30, ax=axs[1])
axs[1].set(xlim=(-1, 1), title="(B) Covariate Distribution, $\\pi_b = 0.4$\nAfter Vector Matching")
axs[1].axvline(x=xmin_loc, color="red", linestyle="--", linewidth=2)
axs[1].axvline(x=xmax_loc, color="red", linestyle="--", linewidth=2)
fig.savefig("../Figures/Appendix/vm.pdf")

## Sigmoidal Sim

In [ ]:
n = 100
p = 10
effect_szs = np.linspace(0, 1, 5)

data_samples = []
data_true = []
for effsz in effect_szs:
    sim = sigmoidal_sim(n, p, eff_sz=effsz, balance=.4, rotate=False)
    Ys = sim["Ys"]; Ts = sim["Ts"]; Xs = sim["Xs"]
    true_t = sim["Ttrue"]; true_y = sim["Ytrue"]; true_x = sim["Xtrue"]
    df = pd.DataFrame({"Group": Ts, "Measurement" : Ys[:,pref], "Covariate" : Xs.flatten(), "Balance":"Unmatched", "Effect Size":effsz})
    df_true = pd.DataFrame({"Group" : true_t, "Measurement" : true_y[:, pref], "Covariate" : true_x, "Effect Size":effsz})

    data_samples.append(df)
    data_true.append(df_true)

data_samples = pd.concat(data_samples)
data_true = pd.concat(data_true)

fig, axs = plt.subplots(1, len(effect_szs), figsize=(15, 3))
for i, effsz in enumerate(effect_szs):
    if i > 0:
        leg = False
    else:
        leg = True
    sns.scatterplot(data=data_samples[data_samples["Effect Size"] == effsz], x="Covariate", y="Measurement", hue="Group", ax=axs[i], alpha=0.5, legend = leg, palette=palette)
    axs[i].set_xlim(-1, 1)
    axs[i].axhline(5, color="red", linewidth=4, linestyle="--")

    sns.lineplot(data=data_true[data_true["Effect Size"] == effsz], x="Covariate", y="Measurement", hue="Group", ax=axs[i], linewidth=3, palette=palette, legend=False)
    axs[i].set_title("$\\Delta$ = {:.2f}".format(effsz))
    axs[i].set_ylabel("Outcome, Dimension 1")
    if i > 0:
        axs[i].set(yticklabels=[], ylabel="")
    axs[i].set_ylim((-2, 12))

fig.savefig("../Figures/Appendix/Sigmoidal.pdf")

## Non-Monotone

In [ ]:
n = 100
p = 10
effect_szs = np.linspace(0, 1, 5)

data_samples = []
data_true = []
for effsz in effect_szs:
    sim = nonmonotone_sim(n, p, eff_sz=effsz, rotate=False, balance=.4)
    Ys = sim["Ys"]; Ts = sim["Ts"]; Xs = sim["Xs"]
    true_t = sim["Ttrue"]; true_y = sim["Ytrue"]; true_x = sim["Xtrue"]
    df = pd.DataFrame({"Group": Ts, "Measurement" : Ys[:,pref], "Covariate" : Xs.flatten(), "Balance":"Unmatched", "Effect Size":effsz})
    df_true = pd.DataFrame({"Group" : true_t, "Measurement" : true_y[:, pref], "Covariate" : true_x, "Effect Size":effsz})

    data_samples.append(df)
    data_true.append(df_true)

data_samples = pd.concat(data_samples)
data_true = pd.concat(data_true)

fig, axs = plt.subplots(1, len(effect_szs), figsize=(15, 3))
for i, effsz in enumerate(effect_szs):
    if i > 0:
        leg = False
    else:
        leg = True
    sns.scatterplot(data=data_samples[data_samples["Effect Size"] == effsz], x="Covariate", y="Measurement", hue="Group", ax=axs[i], alpha=0.5, legend = leg, palette=palette)
    axs[i].set_xlim(-1, 1)

    sns.lineplot(data=data_true[data_true["Effect Size"] == effsz], x="Covariate", y="Measurement", hue="Group", ax=axs[i], linewidth=3, palette=palette, legend=False)
    axs[i].set_title("$\\Delta$ = {:.2f}".format(effsz))
    axs[i].set_ylabel("Outcome, Dimension 1")
    if i > 0:
        axs[i].set(yticklabels=[], ylabel="")
    axs[i].set_ylim((-4, 4))

fig.savefig("../Figures/Appendix/Nonmonotone.pdf")

## K-Class

In [ ]:
n = 100
p = 10
effect_szs = np.linspace(0, 1, 5)

data_samples = []
data_true = []
for effsz in effect_szs:
    sim = kclass_sigmoidal_sim(n, p, eff_sz=effsz, rotate=False, balance=.4)
    Ys = sim["Ys"]; Ts = sim["Ts"]; Xs = sim["Xs"]
    true_t = sim["Ttrue"]; true_y = sim["Ytrue"]; true_x = sim["Xtrue"]
    df = pd.DataFrame({"Group": Ts, "Measurement" : Ys[:,pref], "Covariate" : Xs.flatten(), "Balance":"Unmatched", "Effect Size":effsz})
    df_true = pd.DataFrame({"Group" : true_t, "Measurement" : true_y[:, pref], "Covariate" : true_x, "Effect Size":effsz})

    data_samples.append(df)
    data_true.append(df_true)

data_samples = pd.concat(data_samples)
data_true = pd.concat(data_true)

linestyles={0: "", 1 : "",
         2: (1.5, 3)}

fig, axs = plt.subplots(1, len(effect_szs), figsize=(15, 3))
for i, effsz in enumerate(effect_szs):
    if i > 0:
        leg = False
    else:
        leg = True
    sns.scatterplot(data=data_samples[data_samples["Effect Size"] == effsz], x="Covariate", y="Measurement", hue="Group", ax=axs[i], alpha=0.5, legend = leg, palette=palette)
    axs[i].set_xlim(-1, 1)
    axs[i].axhline(5, color="red", linewidth=4, linestyle="--")

    sns.lineplot(data=data_true[data_true["Effect Size"] == effsz], x="Covariate", y="Measurement", hue="Group", ax=axs[i], linewidth=3, palette=palette, 
                 style="Group", dashes=linestyles, legend=False)
    axs[i].set_title("$\\Delta$ = {:.2f}".format(effsz))
    axs[i].set_ylabel("Outcome, Dimension 1")
    if i > 0:
        axs[i].set(yticklabels=[], ylabel="")
    axs[i].set_ylim((-2, 12))

fig.savefig("../Figures/Appendix/Kclass.pdf")

## Heteroskedastic

In [ ]:
n = 100
p = 10
effect_szs = np.linspace(0, 1, 5)

data_samples = []
data_true = []
for effsz in effect_szs:
    sim = heteroskedastic_sigmoidal_sim(n, p, eff_sz=effsz, rotate=False, balance=.4)
    Ys = sim["Ys"]; Ts = sim["Ts"]; Xs = sim["Xs"]
    true_t = sim["Ttrue"]; true_y = sim["Ytrue"]; true_x = sim["Xtrue"]
    df = pd.DataFrame({"Group": Ts, "Measurement" : Ys[:,pref], "Covariate" : Xs.flatten(), "Balance":"Unmatched", "Effect Size":effsz})
    df_true = pd.DataFrame({"Group" : true_t, "Measurement" : true_y[:, pref], "Covariate" : true_x, "Effect Size":effsz})

    data_samples.append(df)
    data_true.append(df_true)

data_samples = pd.concat(data_samples)
data_true = pd.concat(data_true)

fig, axs = plt.subplots(1, len(effect_szs), figsize=(15, 3))
for i, effsz in enumerate(effect_szs):
    if i > 0:
        leg = False
    else:
        leg = True
    sns.scatterplot(data=data_samples[data_samples["Effect Size"] == effsz], x="Covariate", y="Measurement", hue="Group", ax=axs[i], alpha=0.5, legend = leg, palette=palette)
    axs[i].set_xlim(-1, 1)

    sns.lineplot(data=data_true[data_true["Effect Size"] == effsz], x="Covariate", y="Measurement", hue="Group", ax=axs[i], linewidth=3, palette=palette, legend=False)
    axs[i].set_title("$\\Delta$ = {:.2f}".format(effsz))
    axs[i].set_ylabel("Outcome, Dimension 1")
    if i > 0:
        axs[i].set(yticklabels=[], ylabel="")
    axs[i].set_ylim((-4, 10))

fig.savefig("../Figures/Appendix/Heteroskedastic.pdf")